In [1]:
import os
import sys
import json
import pyspark.sql.functions as f
from pyspark.sql.functions import when, col
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DateType, TimestampType, DecimalType
from pyspark import SparkContext, SparkConf, SQLContext, HiveContext
from collections import namedtuple

In [2]:
!hdfs dfs -ls /user/jairomonassa/covid/excel

Found 4 items
-rw-r--r--   3 root supergroup   62492959 2022-08-08 00:15 /user/jairomonassa/covid/excel/HIST_PAINEL_COVIDBR_2020_Parte1_06jul2021.csv
-rw-r--r--   3 root supergroup   76520681 2022-08-08 00:15 /user/jairomonassa/covid/excel/HIST_PAINEL_COVIDBR_2020_Parte2_06jul2021.csv
-rw-r--r--   3 root supergroup   91120916 2022-08-08 00:15 /user/jairomonassa/covid/excel/HIST_PAINEL_COVIDBR_2021_Parte1_06jul2021.csv
-rw-r--r--   3 root supergroup    3046774 2022-08-08 00:15 /user/jairomonassa/covid/excel/HIST_PAINEL_COVIDBR_2021_Parte2_06jul2021.csv


In [3]:
conf = SparkConf().set("spark.driver.memory", "8G")\
    .set("spark.executor.cores", "5")\
    .set("spark.executor.instances", "5")\
    .set("spark.executor.memory", "10G")\
    .set("spark.sql.files.maxPartitionBytes", "128MB")\
    .set("spark.sql.adaptive.enabled", "true")\
    .set("spark.sql.shuffle.partitions", "50")\
    .set("spark.yarn.executor.memoryOverhead", "2")\
    .set("spark.yarn.driver.memoryOverhead", "1G")

In [4]:
spark = SparkSession.builder \
        .master("yarn") \
        .enableHiveSupport() \
        .getOrCreate()

In [5]:
sql = SQLContext(sc)
hc = HiveContext(sc)

In [6]:
schema = StructType([ \
    StructField("regiao",StringType(),True), \
    StructField("estado",StringType(),True), \
    StructField("municipio",StringType(),True), \
    StructField("coduf", IntegerType(), True), \
    StructField("codmun", IntegerType(), True), \
    StructField("codRegiaoSaude", IntegerType(), True), \
    StructField("nomeRegiaoSaude",StringType(),True), \
    StructField("data",TimestampType(),True), \
    StructField("semanaEpi",IntegerType(),True), \
    StructField("populacaoTCU2019", IntegerType(), True), \
    StructField("casosAcumulado", DecimalType(), True), \
    StructField("casosNovos", IntegerType(), True) ,\
    StructField("obitosAcumulado", IntegerType(), True), \
    StructField("obitosNovos", IntegerType(), True), \
    StructField("Recuperadosnovos", IntegerType(), True), \
    StructField("emAcompanhamentoNovos", IntegerType(), True), \
    StructField("interior/metropolitana", IntegerType(), True), \
  ])

In [7]:
df = spark.read.option("header","true")\
 .option("schema",schema)\
 .option("mode","DROPMALFORMED")\
 .option("delimiter", ";")\
 .option("inferSchema", "true")\
 .csv("/user/jairomonassa/covid/excel/*.csv")

In [8]:
df.count()

2624943

In [9]:
df.printSchema()

root
 |-- regiao: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- coduf: integer (nullable = true)
 |-- codmun: integer (nullable = true)
 |-- codRegiaoSaude: integer (nullable = true)
 |-- nomeRegiaoSaude: string (nullable = true)
 |-- data: timestamp (nullable = true)
 |-- semanaEpi: integer (nullable = true)
 |-- populacaoTCU2019: integer (nullable = true)
 |-- casosAcumulado: decimal(10,0) (nullable = true)
 |-- casosNovos: integer (nullable = true)
 |-- obitosAcumulado: integer (nullable = true)
 |-- obitosNovos: integer (nullable = true)
 |-- Recuperadosnovos: integer (nullable = true)
 |-- emAcompanhamentoNovos: integer (nullable = true)
 |-- interior/metropolitana: integer (nullable = true)



In [10]:
arq1 = !hdfs dfs -cat /user/jairomonassa/covid/excel/HIST_PAINEL_COVIDBR_2020_Parte1_06jul2021.csv | wc -l
arq2 = !hdfs dfs -cat /user/jairomonassa/covid/excel/HIST_PAINEL_COVIDBR_2020_Parte2_06jul2021.csv | wc -l
arq3 = !hdfs dfs -cat /user/jairomonassa/covid/excel/HIST_PAINEL_COVIDBR_2021_Parte1_06jul2021.csv | wc -l
arq4 = !hdfs dfs -cat /user/jairomonassa/covid/excel/HIST_PAINEL_COVIDBR_2021_Parte2_06jul2021.csv | wc -l

In [11]:
#desconsiderar header
arq1 = int(arq1[0])-1
arq2 = int(arq2[0])-1
arq3 = int(arq3[0])-1
arq4 = int(arq4[0])-1
arq_count= arq1 + arq2 + arq3 + arq4

In [12]:
arq_count

2624943

In [13]:
df.show(2)

+------+------+---------+-----+------+--------------+---------------+-------------------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+----------------------+
|regiao|estado|municipio|coduf|codmun|codRegiaoSaude|nomeRegiaoSaude|               data|semanaEpi|populacaoTCU2019|casosAcumulado|casosNovos|obitosAcumulado|obitosNovos|Recuperadosnovos|emAcompanhamentoNovos|interior/metropolitana|
+------+------+---------+-----+------+--------------+---------------+-------------------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+----------------------+
|Brasil|  null|     null|   76|  null|          null|           null|2021-01-01 00:00:00|       53|       210147125|       7700578|     24605|         195411|        462|         6756284|               748883|                  null|
|Brasil|  null|     null|   76|  null|          null|           null

In [14]:
df= df.withColumnRenamed("codRegiaoSaude","codregiaosaude")\
	.withColumnRenamed("nomeRegiaoSaude","nomeregiaosaude")\
	.withColumnRenamed("semanaEpi","semanaepi")\
	.withColumnRenamed("populacaoTCU2019","populacaotcu2019")\
	.withColumnRenamed("casosAcumulado","casosacumulado")\
	.withColumnRenamed("casosNovos","casosnovos")\
	.withColumnRenamed("obitosAcumulado","obitosacumulado")\
	.withColumnRenamed("obitosNovos","obitosnovos")\
	.withColumnRenamed("emAcompanhamentoNovos","emacompanhamentonovos")\
	.withColumnRenamed("interior/metropolitana","interiormetropolitana")

In [19]:
df.write.mode("overwrite").partitionBy("municipio").saveAsTable("covid")

In [20]:
!hdfs dfs -ls /user/hive/warehouse/covid_t

Found 5299 items
-rw-r--r--   2 root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/_SUCCESS
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Abadia de Goiás
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Abadia dos Dourados
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Abadiânia
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Abaetetuba
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Abaeté
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Abaiara
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Abaré
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Abatiá
drwxr-xr-x   - ro

drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Alto Horizonte
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Alto Jequitibá
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Alto Longá
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Alto Paraguai
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Alto Paraná
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Alto Paraíso
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Alto Paraíso de Goiás
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Alto Parnaíba
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Al

drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Baião
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Balbinos
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Baldim
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Baliza
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Balneário Arroio do Silva
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Balneário Barra do Sul
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Balneário Camboriú
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Balneário Gaivota
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Bal

drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Bossoroca
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Botelhos
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Botucatu
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Botumirim
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Botuporã
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Botuverá
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Bozano
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Braga
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Braganey
drwxr-xr-x   - root supergroup          

drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Campo Novo do Parecis
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Campo Redondo
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Campo Verde
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Campo do Brito
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Campo do Meio
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Campo do Tenente
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Campos Altos
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Campos Belos
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipi

...

drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=São Vendelino
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=São Vicente
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=São Vicente Ferrer
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=São Vicente Férrer
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=São Vicente de Minas
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=São Vicente do Seridó
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=São Vicente do Sul
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Sério
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covi

drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Telha
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Telêmaco Borba
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Tenente Ananias
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Tenente Laurentino Cruz
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Tenente Portela
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Tenório
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Teodoro Sampaio
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Teofilândia
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Teolâ

drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Tucuruí
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Tufilândia
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Tuiuti
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Tumiritinga
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Tunas
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Tunas do Paraná
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Tuneiras do Oeste
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Tuntum
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Tunápolis
drwxr-xr-x   - root sup

drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Urucuia
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Urucurituba
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Urucânia
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Uruguaiana
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Uruoca
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Urupema
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Urupá
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Urupês
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Urussanga
drwxr-xr-x   - root supergroup          

drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Viçosa do Ceará
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Volta Grande
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Volta Redonda
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Votorantim
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Votuporanga
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Várzea
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Várzea Alegre
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Várzea Branca
drwxr-xr-x   - root supergroup          0 2022-08-08 23:29 /user/hive/warehouse/covid_t/municipio=Várzea Grande
drw